## Single Module Tests

In [1]:
import numpy as np
import torch

In [2]:
from ddnn.nn import Initializer

Initializer("random_uniform")((4, 5))
Initializer("random_normal")((4, 5))
Initializer("glorot_uniform")((4, 5))
Initializer("glorot_normal")((4, 5))
Initializer("he_uniform", fan_mode="fan_in")((4, 5))
Initializer("he_normal", fan_mode="fan_out")((4, 5));

In [3]:
from ddnn.nn import ActivationFunction


def assert_activation_function(fname, pytorch_equivalent):
    inputs = np.array([0, -1, 1, -1e5, 1e5, 1e-5, -1e-5])
    tinput = torch.Tensor(inputs).requires_grad_()
    afun = ActivationFunction(fname)

    outputs = afun.foward(inputs)
    expected = pytorch_equivalent(tinput)
    assert np.allclose(outputs, expected.detach()), f"{fname} func"

    expected.sum().backward()
    assert np.allclose(
        afun.backward(np.ones_like(outputs)), tinput.grad
    ), f"{fname} grad"


assert_activation_function("ReLU", torch.nn.ReLU())
assert_activation_function("logistic", torch.nn.Sigmoid())
assert_activation_function("tanh", torch.nn.Tanh())
# overflow error in logistic function can be safely ignored

/home/david/uni/ml/Machine-Learning-Project/ddnn/nn/activation.py:86: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [4]:
from ddnn.nn import LossFunction

inputs = np.array([0, 1, 0, 1])
labels = np.array([0, 0, 1, 1])
tinput = torch.Tensor(inputs).requires_grad_()
tlabels = torch.Tensor(labels)

tinput.grad = None
lfun = LossFunction("MSE")
outputs = lfun.foward(inputs.reshape(inputs.size, 1), labels)
expected = torch.nn.MSELoss()(tinput, tlabels)
assert np.allclose(outputs, expected.detach()), "MSE func"
expected.backward()
assert np.allclose(lfun.backward().squeeze(), tinput.grad), "MSE grad"

# TODO add manual check for accuracy and MEE

In [5]:
from ddnn.nn import LinearLayer

tlayer = torch.nn.Linear(10, 2)
layer = LinearLayer((10, 2))
layer.params.weights[:] = tlayer.weight.detach()
layer.params.bias[:] = tlayer.bias.detach()

tinputs = torch.rand(8, 10, requires_grad=True)
tlabels = torch.rand(8, 2)
inputs = tinputs.detach().numpy()
labels = tlabels.detach().numpy()

tpreds = tlayer(tinputs)
preds = layer.foward(inputs)
assert np.allclose(preds, tpreds.detach()), "Linear Forward"

tpreds.sum().backward()
tigrad = tinputs.grad
igrad = layer.backward(np.ones_like(preds))
assert np.allclose(igrad, tigrad.detach()), "Linear Backward [Input]"
assert np.allclose(layer.grads.weights, tlayer.weight.grad), "Linear Backward [Weights]"
assert np.allclose(layer.grads.bias, tlayer.bias.grad), "Linear Backward [Bias]"

## MultiLayer Test
### Pytorch

In [ ]:
tnet = torch.nn.Sequential(
    torch.nn.Linear(8, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 2),
)
tloss_fn = torch.nn.MSELoss()

In [ ]:
tdata = torch.rand(10, 8, requires_grad=True)
tlabel = torch.rand(10, 2)
tpred = tnet(tdata)
tloss = tloss_fn(tpred, tlabel)
tloss

tensor(0.5451, grad_fn=<MseLossBackward0>)

In [ ]:
tloss.backward()
tgrad = tdata.grad

### Ours

In [ ]:
from ddnn.nn import NeuralNetwork, LinearLayer, ActivationFunction, LossFunction

In [ ]:
net = NeuralNetwork(
    [
        LinearLayer((8, 16)),
        ActivationFunction(),
        LinearLayer((16, 16)),
        ActivationFunction(),
        LinearLayer((16, 2)),
    ]
)
loss_fn = LossFunction()

In [ ]:
# We force the same initial weights for both networks
for layer, tlayer in zip(net[:], tnet):
    if isinstance(layer, LinearLayer):
        layer.params.weights[:] = tlayer.weight.detach().numpy()
        layer.params.bias[:] = tlayer.bias.detach().numpy()

In [ ]:
data = tdata.detach().numpy()
label = tlabel.numpy()
pred = net.foward(data)
loss = loss_fn.foward(pred, label)
loss

1.090184360879591

In [ ]:
grad = net.backward(loss_fn.backward())

### Assessment

In [ ]:
# check for correct foward computation by comparing with pytorch
np.allclose(pred, tpred.detach().numpy()), np.linalg.norm(pred - tpred.detach().numpy())

(True, 2.787329340050096e-08)

In [ ]:
# check for correct backward computation by comparing with pytorch
np.allclose(grad, tgrad.numpy()), np.linalg.norm(grad - tgrad.numpy())

(True, 3.0641544645357924e-09)

In [ ]:
for i, (layer, tlayer) in enumerate(zip(net[::-1], tnet[::-1])):
    if isinstance(layer, LinearLayer):
        assert np.allclose(layer.grads.weights, tlayer.weight.grad.detach().numpy()), i
        assert np.allclose(layer.grads.bias, tlayer.bias.grad.detach().numpy()), i